In [23]:
!pip install bng-latlon

Defaulting to user installation because normal site-packages is not writeable
  Created wheel for bng-latlon: filename=bng_latlon-1.0-py3-none-any.whl size=7938 sha256=4b275ab578a4983404bc0be73cc89cf707a7985284f5186e06a1d614720a22c1
  Stored in directory: /home/jupyter/.cache/pip/wheels/c8/25/f8/453f6e014d4801a4e802fbc61283388d7c1c3b651ac2848a16
Successfully built bng-latlon


In [2]:
#import pandas_gbq as pd
from google.cloud import bigquery
from google.oauth2 import service_account
import textwrap
import pandas as pd
from bng_to_latlon import OSGB36toWGS84
import numpy as np

In [5]:
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file(
    '../keys/nr-dr-simulation-phase-2-dev-b758d47719cd.json')
project_id = 'nr-dr-simulation-phase-2-dev'
client = bigquery.Client(credentials= credentials,project=project_id)

In [16]:
dquery_string = 'select * from oda.incidents_assets'

In [17]:
query_job3 = client.query(dquery_string)

In [18]:
dataframe=query_job3.to_dataframe()# 

In [19]:
dataframe.to_pickle('../data/asset_incidents.pickle')

In [20]:
dquery_string2 = 'select * from oda.assetlink'

In [21]:
query_job2 = client.query(dquery_string2)

In [22]:
assets=query_job2.to_dataframe()# 

In [25]:
assets.to_pickle('../data/assets.pickle')

In [7]:
assets= pd.read_pickle('../data/assets.pickle')

In [8]:
assets.columns

Index(['DSTRCT_CODE', 'EQUIP_STATUS', 'ACTIVE_FLG', 'COSTING_FLG', 'ROUTE',
       'DU', 'EQUIP_CLASSIFX3', 'EX3_DESC', 'EQUIP_CLASSIFX15', 'EX15_DESC',
       'ELLIPSE_ASSET_DISCIPLINE_GROUP', 'ELLIPSE_ASSET_MAINT_ENGINEER',
       'ELLIPSE_ASSET_SECTION_MANAGER', 'EQUIP_CLASS', 'EQUIP_CLASS_DESC',
       'EQUIP_CLASS_ACTIVE_FLAG', 'ITEM_NAME_CODE', 'ITEM_NAME_1',
       'ITEM_NAME_2', 'EQUIP_GRP_ID', 'EGI_DESC', 'EGI_ACTIVE_FLAG',
       'EQUIP_NO', 'PARENT_EQUIP', 'PLANT_NO', 'ELR', 'TRACK_REF',
       'MILEAGE_FROM', 'MILEAGE_TO', 'IDENTIFIER', 'ASSET_CREATION_DATE',
       'ASSET_CREATION_USER_DESC', 'ASSET_LAST_MOD_DATE',
       'ASSET_LAST_MOD_USER_DESC', 'Probable_Feature_Type',
       'Start_Easting_centreline', 'Start_Northing_centreline',
       'End_Easting_centreline', 'End_Northing_centreline',
       'Start_Easting_TID', 'Start_Northing_TID', 'End_Easting_TID',
       'End_Northing_TID', 'Manufacturer___Manufacturer', 'YOFI',
       'Grouping_Full_Name', 'Grouping_Key'],

In [9]:
assets.isnull().sum()

DSTRCT_CODE                           0
EQUIP_STATUS                          0
ACTIVE_FLG                            0
COSTING_FLG                           0
ROUTE                                 2
DU                                  271
EQUIP_CLASSIFX3                   75930
EX3_DESC                              0
EQUIP_CLASSIFX15                      0
EX15_DESC                         76474
ELLIPSE_ASSET_DISCIPLINE_GROUP        0
ELLIPSE_ASSET_MAINT_ENGINEER       1869
ELLIPSE_ASSET_SECTION_MANAGER     10165
EQUIP_CLASS                       77763
EQUIP_CLASS_DESC                      0
EQUIP_CLASS_ACTIVE_FLAG               0
ITEM_NAME_CODE                    77763
ITEM_NAME_1                           0
ITEM_NAME_2                           0
EQUIP_GRP_ID                          0
EGI_DESC                              0
EGI_ACTIVE_FLAG                       0
EQUIP_NO                              0
PARENT_EQUIP                      11649
PLANT_NO                              0


In [28]:
def add_columns_long_lat(df, eastings, northings):

    na_count = 0

    for index, row in df.iterrows():

        if pd.isnull(row[eastings]) or pd.isnull(row[northings]):

            na_count += 1

        else:

            long_lat_tuple = OSGB36toWGS84(row[eastings], row[northings])

            long = long_lat_tuple[1]

            lat = long_lat_tuple[0]

            df.at[index,'long'] = long

            df.at[index,'lat'] = lat

    print("Failed Conversions: "+str(na_count))

In [55]:
add_columns_long_lat(assets,"Start_Easting_centreline", "Start_Northing_centreline") # we use Start_Northing_centreline as we just need an aprox location for weather. is not use TID

Failed Conversions: 848


In [32]:
assets[assets.PLANT_NO=='TJC32200 221.0393:221.0393EW02'] # control done with signal coordinates use on SIM Y34038	SIG:L4038

,DSTRCT_CODE,EQUIP_STATUS,ACTIVE_FLG,COSTING_FLG,ROUTE,DU,EQUIP_CLASSIFX3,EX3_DESC,EQUIP_CLASSIFX15,EX15_DESC,...,Start_Easting_TID,Start_Northing_TID,End_Easting_TID,End_Northing_TID,Manufacturer___Manufacturer,YOFI,Grouping_Full_Name,Grouping_Key,long,lat
34156,RTK1,FM,True,A,LNE ...,CC:IMDM Leeds ...,None,Network Rail ...,,None,...,398486.5341,451328.2403,398486.5341,451328.2403,None,NaN,Signalling,S,-2.024553,53.958041


In [34]:
dataframe.columns

Index(['ID', 'Incident_Number', 'Train_ID', 'Incident_Start_Date',
       'Delay_Per_Train', 'Fault_Number', 'Incident_Number_',
       'Engineering_Suffix', 'System_Asset_Type', 'Route', 'Incident_Type',
       'Symptom', 'Failed_Date', 'Occured_Date', 'Rectified_Date',
       'In_Order_Date', 'journey_id', 'Asset_Number', 'ELR',
       'EQUIP_CLASS_DESC'],
      dtype='object')

In [3]:
dataframe=pd.read_csv('./data/tables_fms.csv') # same outcome using jaili table than reading from oda.incidents_assets

In [13]:
len(dataframe)

102902

In [14]:
dataframe.head()

,ID,Incident_Number,Train_ID,Incident_Start_Date,Delay_Per_Train,Fault_Number,Incident_Number_,Engineering_Suffix,System_Asset_Type,Route,Incident_Type,Symptom,Failed_Date,Occured_Date,Rectified_Date,In_Order_Date,journey_id,Asset_Number,ELR,EQUIP_CLASS_DESC
0,3278847,117672,461T23MG17,17/6/2018 07:00:00,12.0,717748,117672,TRACK (P.W),TRACK,LNE ROUTE,0,ASSET DEFECT,2018-06-17 05:16:00 UTC,2018-06-17 00:00:00 UTC,2018-06-17 14:15:00 UTC,2018-06-17 14:15:00 UTC,461T23MG1720180617,48075,DOL2,Track ID ...
1,3281408,117672,171A15MC17,17/6/2018 07:00:00,10.0,717748,117672,TRACK (P.W),TRACK,LNE ROUTE,0,ASSET DEFECT,2018-06-17 05:16:00 UTC,2018-06-17 00:00:00 UTC,2018-06-17 14:15:00 UTC,2018-06-17 14:15:00 UTC,171A15MC1720180617,48075,DOL2,Track ID ...
2,79500,153143,631D62MW02,2/6/2017 22:10:00,21.0,672913,153143,TRACK (P.W),TRACK,LNE ROUTE,NaN,ASSET DEFECT,2017-06-02 21:46:00 UTC,2017-06-02 00:00:00 UTC,2017-06-02 23:53:00 UTC,2017-06-02 23:53:00 UTC,631D62MW0220170206,48081,DOL2,Track ID ...
3,82771,153143,232B36M802,2/6/2017 22:10:00,18.0,672913,153143,TRACK (P.W),TRACK,LNE ROUTE,NaN,ASSET DEFECT,2017-06-02 21:46:00 UTC,2017-06-02 00:00:00 UTC,2017-06-02 23:53:00 UTC,2017-06-02 23:53:00 UTC,232B36M80220170206,48081,DOL2,Track ID ...
4,85318,153143,541D33M702,2/6/2017 22:10:00,21.0,672913,153143,TRACK (P.W),TRACK,LNE ROUTE,NaN,ASSET DEFECT,2017-06-02 21:46:00 UTC,2017-06-02 00:00:00 UTC,2017-06-02 23:53:00 UTC,2017-06-02 23:53:00 UTC,541D33M70220170206,48081,DOL2,Track ID ...


In [16]:
dataframe.isnull().sum()

ID                         0
Incident_Number            0
Train_ID                   0
Incident_Start_Date        0
Delay_Per_Train            0
Fault_Number               0
Incident_Number_           0
Engineering_Suffix         0
System_Asset_Type          0
Route                      0
Incident_Type          72758
Symptom                   30
Failed_Date                0
Occured_Date               0
Rectified_Date           782
In_Order_Date            192
journey_id                 0
Asset_Number               0
ELR                        0
EQUIP_CLASS_DESC           0
dtype: int64

In [21]:
dataframe.Failed_Date.min(), dataframe.Failed_Date.max()

('2017-01-01 07:05:57 UTC', '2018-06-28 20:09:00 UTC')

In [20]:
dataframe.In_Order_Date.unique()

array(['2018-06-17 14:15:00 UTC', '2017-06-02 23:53:00 UTC',
       '2017-06-19 19:07:00 UTC', ..., '2018-01-29 17:18:39 UTC',
       '2018-05-13 23:23:00 UTC', '2018-01-23 14:15:00 UTC'], dtype=object)

In [10]:
assets.columns

Index(['DSTRCT_CODE', 'EQUIP_STATUS', 'ACTIVE_FLG', 'COSTING_FLG', 'ROUTE',
       'DU', 'EQUIP_CLASSIFX3', 'EX3_DESC', 'EQUIP_CLASSIFX15', 'EX15_DESC',
       'ELLIPSE_ASSET_DISCIPLINE_GROUP', 'ELLIPSE_ASSET_MAINT_ENGINEER',
       'ELLIPSE_ASSET_SECTION_MANAGER', 'EQUIP_CLASS', 'EQUIP_CLASS_DESC',
       'EQUIP_CLASS_ACTIVE_FLAG', 'ITEM_NAME_CODE', 'ITEM_NAME_1',
       'ITEM_NAME_2', 'EQUIP_GRP_ID', 'EGI_DESC', 'EGI_ACTIVE_FLAG',
       'EQUIP_NO', 'PARENT_EQUIP', 'PLANT_NO', 'ELR', 'TRACK_REF',
       'MILEAGE_FROM', 'MILEAGE_TO', 'IDENTIFIER', 'ASSET_CREATION_DATE',
       'ASSET_CREATION_USER_DESC', 'ASSET_LAST_MOD_DATE',
       'ASSET_LAST_MOD_USER_DESC', 'Probable_Feature_Type',
       'Start_Easting_centreline', 'Start_Northing_centreline',
       'End_Easting_centreline', 'End_Northing_centreline',
       'Start_Easting_TID', 'Start_Northing_TID', 'End_Easting_TID',
       'End_Northing_TID', 'Manufacturer___Manufacturer', 'YOFI',
       'Grouping_Full_Name', 'Grouping_Key'],

In [56]:
cols=[ 'ROUTE','DU', 'ITEM_NAME_1','ITEM_NAME_2', 'EGI_DESC',  'EQUIP_NO',  'TRACK_REF',
       'MILEAGE_FROM', 'MILEAGE_TO', 'IDENTIFIER',   'Grouping_Full_Name', 'long', 'lat']

In [57]:
assets['EQUIP_NO']=assets['EQUIP_NO'].astype(int)


In [105]:
dataframe['Asset_Number']=dataframe['Asset_Number'].astype(int)

In [12]:
len(dataframe['Asset_Number'].unique())

737

In [106]:
df=pd.merge(dataframe, assets[cols], left_on=['Asset_Number'], right_on= ['EQUIP_NO'], how='inner')
data2=pd.merge(dataframe, assets[cols], left_on=['Asset_Number'], right_on= ['EQUIP_NO'], how='left')

In [107]:
len(df), len(data2)

(102902, 102902)

In [108]:
df.columns

Index(['ID', 'Incident_Number', 'Train_ID', 'Incident_Start_Date',
       'Delay_Per_Train', 'Fault_Number', 'Incident_Number_',
       'Engineering_Suffix', 'System_Asset_Type', 'Route', 'Incident_Type',
       'Symptom', 'Failed_Date', 'Occured_Date', 'Rectified_Date',
       'In_Order_Date', 'journey_id', 'Asset_Number', 'ELR',
       'EQUIP_CLASS_DESC', 'ROUTE', 'DU', 'ITEM_NAME_1', 'ITEM_NAME_2',
       'EGI_DESC', 'EQUIP_NO', 'TRACK_REF', 'MILEAGE_FROM', 'MILEAGE_TO',
       'IDENTIFIER', 'Grouping_Full_Name', 'long', 'lat'],
      dtype='object')

In [109]:
df.to_pickle('../data/incidents_assets_ll.pickle')

In [49]:
df= pd.read_pickle('../data/incidents_assets_ll.pickle')

In [50]:
df.drop([ 'Train_ID', 'Incident_Start_Date','Delay_Per_Train',  'Incident_Number_', 'Incident_Type',
       'Symptom', 'Occured_Date', 'Rectified_Date', 'journey_id', 'DU', 'ITEM_NAME_1', 'ITEM_NAME_2',
        'EQUIP_NO', 'IDENTIFIER'], axis=1, inplace=True)

In [51]:
df.isnull().sum()

ID                      0
Incident_Number         0
Fault_Number            0
Engineering_Suffix      0
System_Asset_Type       0
Route                   0
Failed_Date             0
In_Order_Date         192
Asset_Number            0
ELR                     0
EQUIP_CLASS_DESC        0
ROUTE                   0
EGI_DESC                0
TRACK_REF               0
MILEAGE_FROM            0
MILEAGE_TO              0
Grouping_Full_Name      0
long                    0
lat                     0
dtype: int64

In [52]:
df.rename(columns={'Failed_Date': 'occurred_datetime'}, inplace=True)
df.rename(columns={'In_Order_Date': 'in_order_date'}, inplace=True)

In [53]:
df.head()

,ID,Incident_Number,Fault_Number,Engineering_Suffix,System_Asset_Type,Route,occurred_datetime,in_order_date,Asset_Number,ELR,EQUIP_CLASS_DESC,ROUTE,EGI_DESC,TRACK_REF,MILEAGE_FROM,MILEAGE_TO,Grouping_Full_Name,long,lat
0,3278847,117672,717748,TRACK (P.W),TRACK,LNE ROUTE,2018-06-17 05:16:00 UTC,2018-06-17 14:15:00 UTC,48075,DOL2,Track ID ...,Linear Asset ...,Track ...,1100,175.0704,185.0544,Track,-1.499726,53.676939
1,3281408,117672,717748,TRACK (P.W),TRACK,LNE ROUTE,2018-06-17 05:16:00 UTC,2018-06-17 14:15:00 UTC,48075,DOL2,Track ID ...,Linear Asset ...,Track ...,1100,175.0704,185.0544,Track,-1.499726,53.676939
2,79500,153143,672913,TRACK (P.W),TRACK,LNE ROUTE,2017-06-02 21:46:00 UTC,2017-06-02 23:53:00 UTC,48081,DOL2,Track ID ...,Linear Asset ...,Track ...,2100,175.0704,185.0543,Track,-1.499726,53.676939
3,82771,153143,672913,TRACK (P.W),TRACK,LNE ROUTE,2017-06-02 21:46:00 UTC,2017-06-02 23:53:00 UTC,48081,DOL2,Track ID ...,Linear Asset ...,Track ...,2100,175.0704,185.0543,Track,-1.499726,53.676939
4,85318,153143,672913,TRACK (P.W),TRACK,LNE ROUTE,2017-06-02 21:46:00 UTC,2017-06-02 23:53:00 UTC,48081,DOL2,Track ID ...,Linear Asset ...,Track ...,2100,175.0704,185.0543,Track,-1.499726,53.676939


In [54]:
df.columns

Index(['ID', 'Incident_Number', 'Fault_Number', 'Engineering_Suffix',
       'System_Asset_Type', 'Route', 'occurred_datetime', 'in_order_date',
       'Asset_Number', 'ELR', 'EQUIP_CLASS_DESC', 'ROUTE', 'EGI_DESC',
       'TRACK_REF', 'MILEAGE_FROM', 'MILEAGE_TO', 'Grouping_Full_Name', 'long',
       'lat'],
      dtype='object')

In [55]:
# Transform str to datetime and create extra day columns to remove duplicated fault entries
df['occurred_datetime_1'] = pd.to_datetime(df['occurred_datetime'], errors = 'coerce').dt.date
df['in_order_date_1'] = pd.to_datetime(df['in_order_date'], errors = 'coerce').dt.date
df['occurred_datetime'] = pd.to_datetime(df['occurred_datetime'], errors = 'coerce')
df['in_order_date'] = pd.to_datetime(df['in_order_date'], errors = 'coerce')

In [56]:
len(df)

102902

In [57]:
df = df.drop_duplicates(subset = ['Asset_Number','occurred_datetime_1', 'in_order_date_1','occurred_datetime', 'in_order_date'])
#df = df.drop_duplicates(subset = ['Asset_Number','in_order_date_1'])

In [58]:
#  df = df.drop_duplicates(subset=['ID', 'Incident_Number', 'Fault_Number', 'Engineering_Suffix',
#        'System_Asset_Type', 'Route', 'occurred_datetime', 'in_order_date',
#        'Asset_Number', 'ELR', 'EQUIP_CLASS_DESC', 'ROUTE', 'EGI_DESC',
#        'TRACK_REF', 'MILEAGE_FROM', 'MILEAGE_TO', 'Grouping_Full_Name', 'long',
#        'lat'])

In [59]:
len(df)

1148

In [60]:
# Reset dataframe index
df = df.reset_index(drop=True)

In [33]:
# Transforming 'asset_id' from float to int
df['Asset_Number']=df['Asset_Number'].astype(int)

In [61]:
df['occurred_datetime']=pd.to_datetime(df['occurred_datetime'],utc=True)
df['occurred_datetime'] =df['occurred_datetime'].dt.tz_localize(None)
df['in_order_date']=pd.to_datetime(df['in_order_date'],utc=True)
df['in_order_date'] =df['in_order_date'].dt.tz_localize(None)

In [62]:
df = df.sort_values(by=['Asset_Number','occurred_datetime'])
df.head()

,ID,Incident_Number,Fault_Number,Engineering_Suffix,System_Asset_Type,Route,occurred_datetime,in_order_date,Asset_Number,ELR,...,ROUTE,EGI_DESC,TRACK_REF,MILEAGE_FROM,MILEAGE_TO,Grouping_Full_Name,long,lat,occurred_datetime_1,in_order_date_1
0,3278847,117672,717748,TRACK (P.W),TRACK,LNE ROUTE,2018-06-17 05:16:00,2018-06-17 14:15:00,48075,DOL2,...,Linear Asset ...,Track ...,1100,175.0704,185.0544,Track,-1.499726,53.676939,2018-06-17,2018-06-17
5,3202032,39221,666402,TRACK (P.W),TRACK,LNE ROUTE,2017-04-11 17:34:26,2017-04-11 19:55:01,48081,DOL2,...,Linear Asset ...,Track ...,2100,175.0704,185.0543,Track,-1.499726,53.676939,2017-04-11,2017-04-11
6,3078083,41379,666548,TRACK (P.W),TRACK,LNE ROUTE,2017-04-12 14:54:41,2017-04-23 06:00:01,48081,DOL2,...,Linear Asset ...,Track ...,2100,175.0704,185.0543,Track,-1.499726,53.676939,2017-04-12,2017-04-23
7,4320270,133293,671880,TRACK (P.W),TRACK,LNE ROUTE,2017-05-25 15:10:51,2017-05-26 07:50:48,48081,DOL2,...,Linear Asset ...,Track ...,2100,175.0704,185.0543,Track,-1.499726,53.676939,2017-05-25,2017-05-26
1,79500,153143,672913,TRACK (P.W),TRACK,LNE ROUTE,2017-06-02 21:46:00,2017-06-02 23:53:00,48081,DOL2,...,Linear Asset ...,Track ...,2100,175.0704,185.0543,Track,-1.499726,53.676939,2017-06-02,2017-06-02


In [64]:
df.isnull().sum()

ID                     0
Incident_Number        0
Fault_Number           0
Engineering_Suffix     0
System_Asset_Type      0
Route                  0
occurred_datetime      0
in_order_date          8
Asset_Number           0
ELR                    0
EQUIP_CLASS_DESC       0
ROUTE                  0
EGI_DESC               0
TRACK_REF              0
MILEAGE_FROM           0
MILEAGE_TO             0
Grouping_Full_Name     0
long                   0
lat                    0
occurred_datetime_1    0
in_order_date_1        8
dtype: int64

In [65]:
df.in_order_date.min(), df.in_order_date.max()

(Timestamp('2017-01-01 09:00:01'), Timestamp('2019-02-08 06:12:00'))

In [66]:
df.occurred_datetime.min(), df.occurred_datetime.max()

(Timestamp('2017-01-01 07:05:57'), Timestamp('2018-06-28 20:09:00'))

In [67]:
# Reset dataframe index
df = df.reset_index(drop=True)

In [69]:
df['birth'] = df['death'] = np.nan

In [70]:
# Populate Birth and Death columns based on failure and repair date
for i in range (0,len(df)):
    df.loc[i,'birth'] = df.loc[i,'in_order_date']
    
for i in range (0,len(df)-1):
    if df.loc[i,'Asset_Number'] == df.loc[i+1,'Asset_Number']:
        df.loc[i,'death'] = df.loc[i+1,'occurred_datetime']

In [71]:
df.head()

,ID,Incident_Number,Fault_Number,Engineering_Suffix,System_Asset_Type,Route,occurred_datetime,in_order_date,Asset_Number,ELR,...,TRACK_REF,MILEAGE_FROM,MILEAGE_TO,Grouping_Full_Name,long,lat,occurred_datetime_1,in_order_date_1,birth,death
0,3278847,117672,717748,TRACK (P.W),TRACK,LNE ROUTE,2018-06-17 05:16:00,2018-06-17 14:15:00,48075,DOL2,...,1100,175.0704,185.0544,Track,-1.499726,53.676939,2018-06-17,2018-06-17,2018-06-17 14:15:00,NaN
1,3202032,39221,666402,TRACK (P.W),TRACK,LNE ROUTE,2017-04-11 17:34:26,2017-04-11 19:55:01,48081,DOL2,...,2100,175.0704,185.0543,Track,-1.499726,53.676939,2017-04-11,2017-04-11,2017-04-11 19:55:01,2017-04-12 14:54:41
2,3078083,41379,666548,TRACK (P.W),TRACK,LNE ROUTE,2017-04-12 14:54:41,2017-04-23 06:00:01,48081,DOL2,...,2100,175.0704,185.0543,Track,-1.499726,53.676939,2017-04-12,2017-04-23,2017-04-23 06:00:01,2017-05-25 15:10:51
3,4320270,133293,671880,TRACK (P.W),TRACK,LNE ROUTE,2017-05-25 15:10:51,2017-05-26 07:50:48,48081,DOL2,...,2100,175.0704,185.0543,Track,-1.499726,53.676939,2017-05-25,2017-05-26,2017-05-26 07:50:48,2017-06-02 21:46:00
4,79500,153143,672913,TRACK (P.W),TRACK,LNE ROUTE,2017-06-02 21:46:00,2017-06-02 23:53:00,48081,DOL2,...,2100,175.0704,185.0543,Track,-1.499726,53.676939,2017-06-02,2017-06-02,2017-06-02 23:53:00,2017-06-17 12:12:00


In [41]:
df['birth']=pd.to_datetime(df['birth'], format='%d/%m/%Y %H:%M:%S')
df['death']=pd.to_datetime(df['death'], format='%d/%m/%Y %H:%M:%S')
df['birth'] = df['birth'].apply(lambda d: d.replace(tzinfo=None))
df['death'] = df['death'].apply(lambda d: d.replace(tzinfo=None))

In [42]:
df['occurred_datetime']=pd.to_datetime(df['occurred_datetime'], format='%d/%m/%Y %H:%M:%S')
df['in_order_date']=pd.to_datetime(df['in_order_date'], format='%d/%m/%Y %H:%M:%S')
df['occurred_datetime'] = df['occurred_datetime'].apply(lambda d: d.replace(tzinfo=None))
df['in_order_date'] = df['in_order_date'].apply(lambda d: d.replace(tzinfo=None))

In [43]:
df.head()

,ID,Incident_Number,Fault_Number,Engineering_Suffix,System_Asset_Type,Route,occurred_datetime,in_order_date,Asset_Number,ELR,...,TRACK_REF,MILEAGE_FROM,MILEAGE_TO,Grouping_Full_Name,long,lat,occurred_datetime_1,in_order_date_1,birth,death
0,3278847,117672,717748,TRACK (P.W),TRACK,LNE ROUTE,2018-06-17 05:16:00,2018-06-17 14:15:00,48075,DOL2,...,1100,175.0704,185.0544,Track,-1.499726,53.676939,2018-06-17,2018-06-17,2018-06-17 14:15:00,NaT
1,3202032,39221,666402,TRACK (P.W),TRACK,LNE ROUTE,2017-04-11 17:34:26,2017-04-11 19:55:01,48081,DOL2,...,2100,175.0704,185.0543,Track,-1.499726,53.676939,2017-04-11,2017-04-11,2017-04-11 19:55:01,2017-04-12 14:54:41
2,3078083,41379,666548,TRACK (P.W),TRACK,LNE ROUTE,2017-04-12 14:54:41,2017-04-23 06:00:01,48081,DOL2,...,2100,175.0704,185.0543,Track,-1.499726,53.676939,2017-04-12,2017-04-23,2017-04-23 06:00:01,2017-05-25 15:10:51
3,4320270,133293,671880,TRACK (P.W),TRACK,LNE ROUTE,2017-05-25 15:10:51,2017-05-26 07:50:48,48081,DOL2,...,2100,175.0704,185.0543,Track,-1.499726,53.676939,2017-05-25,2017-05-26,2017-05-26 07:50:48,2017-06-02 21:46:00
4,79500,153143,672913,TRACK (P.W),TRACK,LNE ROUTE,2017-06-02 21:46:00,2017-06-02 23:53:00,48081,DOL2,...,2100,175.0704,185.0543,Track,-1.499726,53.676939,2017-06-02,2017-06-02,2017-06-02 23:53:00,2017-06-17 12:12:00


In [76]:
df.isnull().sum()

ID                       0
Incident_Number          0
Fault_Number             0
Engineering_Suffix       0
System_Asset_Type        0
Route                    0
occurred_datetime        0
in_order_date            8
Asset_Number             0
ELR                      0
EQUIP_CLASS_DESC         0
ROUTE                    0
EGI_DESC                 0
TRACK_REF                0
MILEAGE_FROM             0
MILEAGE_TO               0
Grouping_Full_Name       0
long                     0
lat                      0
occurred_datetime_1      0
in_order_date_1          8
birth                    8
death                  737
timecheck                0
dtype: int64

In [77]:
len(df.Asset_Number.unique())

737

In [78]:
len(df)

1148

In [72]:
df.birth.max(), df.death.max()

TypeError: unorderable types: Timestamp() >= float()

In [73]:
df['timecheck']=df['birth'] > df['death']

In [74]:
df['timecheck'].unique()

array([False,  True])

In [75]:
df.loc[df['timecheck'] == True, ['Asset_Number','occurred_datetime','in_order_date']]

,Asset_Number,occurred_datetime,in_order_date
15,48358,2017-04-10 19:12:57,2017-07-02 08:00:01
53,51864,2017-03-13 22:25:06,2017-09-04 05:00:01
62,53991,2018-02-12 03:30:00,2018-02-18 07:00:00
74,53999,2018-04-19 13:17:00,2018-04-24 18:00:00
101,54014,2018-05-07 22:05:00,2018-06-06 11:35:00
113,54035,2017-12-11 21:24:00,2018-05-06 02:32:00
119,54035,2018-02-08 21:43:00,2018-03-07 08:47:00
120,54035,2018-02-08 21:45:00,2018-03-07 08:49:00
123,54035,2018-05-06 10:38:00,2018-05-07 00:13:00
125,54035,2018-05-25 05:33:00,2019-01-09 15:00:00
